# Load Dataset

Load the dataset of the Mars Express Power Challenge

## import libraries

In [1]:
!{sys.executable} -m pip install pandas

import pandas as pd
from datetime import datetime
from pandas import Timestamp

## Data Download

In [26]:
%run download_dataset.ipynb

In [6]:
# we donwload the dataset and unzip it inside the folder
download_unzip(url, file_name + file_extension, folder)

## Data Loading

The credit for most of the code in this section goes to Alexander Bauer, who very kindly released a baseline script for the competition on github:

* https://github.com/alex-bauer/kelvin-power-challenge
* [Kelvins - Discussions / Tutorial: SAAF+LTDATA Random Forest baseline script (0.12 PLB)](https://kelvins.esa.int/mars-express-power-challenge/discussion/56/)

Specifically, it includes the code from his [rf_baseline.py](https://github.com/alex-bauer/kelvin-power-challenge/blob/a968bdb7f759e35b7f0076d794f6cc517c0dd1b3/src/rf_baseline.py) file.

### Helper Functions

In [27]:
# Function to convert the utc timestamp to datetime
def convert_time(df):
    df['ut_ms'] = pd.to_datetime(df['ut_ms'], unit='ms')
    return df

# Function to resample the dataframe to hourly mean
def resample_1H(df):
    df = df.set_index('ut_ms')
    df = df.resample('1H').mean()
    return df

# Function to read a csv file and resample to hourly consumption
def parse_ts(filename, dropna=True):
    df = pd.read_csv(PATH_TO_DATA + '/' + filename)
    df = convert_time(df)
    df = resample_1H(df)
    if dropna:
        df = df.dropna()
    return df

# Function to read the ltdata files
def parse_ltdata(filename):
    df = pd.read_csv(PATH_TO_DATA + '/' + filename)
    df = convert_time(df)
    df = df.set_index('ut_ms')
    return df

**Load the power files**

In [28]:
## Load the power files: they are the columns that need to be predicted (predicted value)
pow_train1 = parse_ts('/train_set/power--2008-08-22_2010-07-10.csv')
pow_train2 = parse_ts('/train_set/power--2010-07-10_2012-05-27.csv')
pow_train3 = parse_ts('/train_set/power--2012-05-27_2014-04-14.csv')

# Load the test sample submission file as template for prediction
pow_test = parse_ts('power-prediction-sample-2014-04-14_2016-03-01.csv', False)

# Concatenate the files
power_all = pd.concat([pow_train1, pow_train2, pow_train3, pow_test])

/home/cguz/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  


In [9]:
# Extract the columns that need to be predicted
power_cols = list(power_all.columns)
print('power cols')
print(' '.join(power_cols))

NPWD2372 NPWD2401 NPWD2402 NPWD2451 NPWD2471 NPWD2472 NPWD2481 NPWD2482 NPWD2491 NPWD2501 NPWD2531 NPWD2532 NPWD2551 NPWD2552 NPWD2561 NPWD2562 NPWD2691 NPWD2721 NPWD2722 NPWD2742 NPWD2771 NPWD2791 NPWD2792 NPWD2801 NPWD2802 NPWD2821 NPWD2851 NPWD2852 NPWD2871 NPWD2872 NPWD2881 NPWD2882 NPWD2692


**Load SAAF files**

In [10]:
# Load the saaf files
saaf_train1 = parse_ts('/train_set/context--2008-08-22_2010-07-10--saaf.csv')
saaf_train2 = parse_ts('/train_set/context--2010-07-10_2012-05-27--saaf.csv')
saaf_train3 = parse_ts('/train_set/context--2012-05-27_2014-04-14--saaf.csv')
saaf_test = parse_ts('/test_set/context--2014-04-14_2016-03-01--saaf.csv')
saaf_all = pd.concat([saaf_train1, saaf_train2, saaf_train3, saaf_test])

In [11]:
# Extract the columns name
saaf_cols = list(saaf_all.columns)
print('saaf cols')
print(' '.join(saaf_cols))

sa sx sy sz


**Load LTDATA files**

In [12]:
# Load the ltdata files
ltdata_train1 = parse_ltdata('/train_set/context--2008-08-22_2010-07-10--ltdata.csv')
ltdata_train2 = parse_ltdata('/train_set/context--2010-07-10_2012-05-27--ltdata.csv')
ltdata_train3 = parse_ltdata('/train_set/context--2012-05-27_2014-04-14--ltdata.csv')
ltdata_test = parse_ltdata('/test_set/context--2014-04-14_2016-03-01--ltdata.csv')
ltdata_all = pd.concat([ltdata_train1, ltdata_train2, ltdata_train3, ltdata_test])

In [13]:
# Extract the columns name
ltdata_cols = list(ltdata_all.columns)
print('ltdata cols')
print(' '.join(ltdata_cols))

sunmars_km earthmars_km sunmarsearthangle_deg solarconstantmars eclipseduration_min occultationduration_min


**Load DMOP files**

In [14]:
def parse_dmop(filename):
    df = pd.read_csv(PATH_TO_DATA + '/' + filename)
    df = convert_time(df)

    subsystems = sorted({s[1:4] for s in df['subsystem'] if s[0] == 'A'})
    
    df_expansion = [
        [when] + [(1 if cmd[1:4]==syst else 0) for syst in subsystems]
        for (when, cmd) in df.values
        if cmd[0]=='A']
    
    df = pd.DataFrame(df_expansion, columns=['ut_ms'] + subsystems)
    df = df.set_index('ut_ms')
    
    # get one row per hour, containing the boolean values indicating whether each
    # subsystem was activated in that hour;
    # hours not represented in the file have all columns with 0.
    # df = df.resample('1H').max().fillna(0)
    # cells represent number of times intructions were issued to that subsystem in that hour
    df = df.resample('1H').sum().fillna(0)
    
    return df

In [15]:
# Load the dmop files
dmop_train1 = parse_dmop('/train_set/context--2008-08-22_2010-07-10--dmop.csv')
dmop_train2 = parse_dmop('/train_set/context--2010-07-10_2012-05-27--dmop.csv')
dmop_train3 = parse_dmop('/train_set/context--2012-05-27_2014-04-14--dmop.csv')
dmop_test = parse_dmop('/test_set/context--2014-04-14_2016-03-01--dmop.csv')
dmop_all = pd.concat([dmop_train1, dmop_train2, dmop_train3, dmop_test]).fillna(0)

In [16]:
# Extract the columns name
dmop_cols = list(dmop_all.columns)
print('dmop cols')
print(' '.join(dmop_cols))

AAA HHH MMM OOO PSF PWF SEQ SSS SXX TMB TTT VVV XXX ACF DMC DMF


**Load FTL files**

In [17]:
# Create df to join everything together
df = power_all

In [18]:
def parse_ftl(filename):
    df = pd.read_csv(PATH_TO_DATA + '/' + filename)
    df['utb_ms'] = pd.to_datetime(df['utb_ms'], unit='ms')
    df['ute_ms'] = pd.to_datetime(df['ute_ms'], unit='ms')
    return df

def parse_ftl_all(filenames, hour_indices):
    ftl_all = pd.concat([parse_ftl(f) for f in filenames])
    
    types = sorted(set(ftl_all['type']))
    ftl_df = pd.DataFrame(index=hour_indices, columns=['flagcomms'] + types).fillna(0)
    
    # hour indices of discarded events, because of non-ocurrence in `hour_indices`
    ix_err = []

    for (t_start, t_end, p_type, comms) in ftl_all.values:
        floor_beg = Timestamp(t_start).floor('1h')
        floor_end = Timestamp(t_end).floor('1h')
        
        try:
            ftl_df.loc[floor_beg]['flagcomms'] = ftl_df.loc[floor_end]['flagcomms'] = int(comms)
            ftl_df.loc[floor_beg][p_type]      = ftl_df.loc[floor_end][p_type]      = 1
        except KeyError:
            ix_err.append((floor_beg, floor_end))
    
    print('Warning: discarded %d FTL events' % len(ix_err))
    
    return ftl_all, ftl_df

In [19]:
ftl_fnames = [
    '/train_set/context--2008-08-22_2010-07-10--ftl.csv',
    '/train_set/context--2010-07-10_2012-05-27--ftl.csv',
    '/train_set/context--2012-05-27_2014-04-14--ftl.csv',
    '/test_set/context--2014-04-14_2016-03-01--ftl.csv',
    ]
%time ftl_all, ftl_df = parse_ftl_all(filenames=ftl_fnames, hour_indices=df.index)

CPU times: user 53.3 s, sys: 30.9 ms, total: 53.3 s
Wall time: 53.3 s


In [20]:
# Extract the columns name
ftl_cols = list(ftl_all.columns)
print('ftl cols')
print(' '.join(ftl_cols))

utb_ms ute_ms type flagcomms


**Join cols names**

In [21]:
all_cols = saaf_cols + ltdata_cols + dmop_cols + ftl_cols

In [22]:
# print(all_cols)

['sa', 'sx', 'sy', 'sz', 'sunmars_km', 'earthmars_km', 'sunmarsearthangle_deg', 'solarconstantmars', 'eclipseduration_min', 'occultationduration_min', 'AAA', 'HHH', 'MMM', 'OOO', 'PSF', 'PWF', 'SEQ', 'SSS', 'SXX', 'TMB', 'TTT', 'VVV', 'XXX', 'ACF', 'DMC', 'DMF', 'utb_ms', 'ute_ms', 'type', 'flagcomms']
